This notebook aims to find the best ordering of targets, from a random subset of permutations

new objective: get the predictions of all chains and average them out and measure the performance -- Does an ensemble of chains lead to a clear improvement over just one chain?

In [1]:
import numpy as np
import pandas as pd
import itertools
import random

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score
from chaining import Chain
import os
from scipy.stats import mode

In [2]:
def generate_permutations_with_order(variables, pairs_or_groups, num_permutations, constrained_elements=None, random_state=None):
    random.seed(random_state)
    permutations_list = [list(variables)]  # Add the original order only once as a list of strings
    
    while len(permutations_list) < num_permutations + 1: #+1 because the origin order is counted too
        perm = list(random.sample(variables, len(variables)))
        valid = True
        
        for pair_or_group in pairs_or_groups:
            idxs = [perm.index(var) for var in pair_or_group]
            if sorted(idxs) != idxs:
                valid = False
                break
        
        if valid:
            if constrained_elements:
            # Check if all constrained elements are present in the first positions of the permutation
                if all(elem in perm[:len(constrained_elements)] for elem in constrained_elements):
                    permutations_list.append(perm)
        
    
    return permutations_list

In [3]:
# TRY 2
def generate_permutations_with_order_2(variables, pairs_or_groups, num_permutations, constrained_elements=None, random_state=None):
    random.seed(random_state)
    
    # Generate all permutations of the variables
    permutations = itertools.permutations(variables)
    
    # Filter permutations according to the specified pairs or groups
    ordered_permutations = []
    for perm in permutations:
        valid = True
        for pair_or_group in pairs_or_groups:
            idxs = [perm.index(var) for var in pair_or_group]
            if sorted(idxs) != idxs:
                valid = False
                break
        if valid:
            ordered_permutations.append(perm)
    
    # If constrained elements are specified, ensure they appear at the beginning of each permutation
    if constrained_elements:
        constrained_permutations = []
        for perm in ordered_permutations:
            # Check if all constrained elements are present in the first positions of the permutation
            if all(elem in perm[:len(constrained_elements)] for elem in constrained_elements):
                constrained_permutations.append(perm)
        ordered_permutations = constrained_permutations
    
    # Select random permutations from the filtered permutations
    random_permutations = random.sample(ordered_permutations, min(num_permutations, len(ordered_permutations)))
    
    # Convert permutations to list of strings
    permutations_list = [list(variables)]  # Add the original order only once as a list of strings
    for perm in random_permutations:
        permutations_list.append(list(perm))  # Convert each permutation to a list of strings
    
    return permutations_list

In [4]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [5]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# Define file names
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)

data = pd.read_csv(path)

def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

data

,USUBJID,AGE,SEX,RACE,CONTINENT,CESEV,CECONTRT,TOTRELAP,MHCONTRT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,SMSTDY,NRELAP,NHPT-before,NHPT-2y,NHPT-after_2y,PASAT_2s-before,PASAT_2s-2y,PASAT_2s-after_2y,PASAT_3s-before,PASAT_3s-2y,PASAT_3s-after_2y,SDMT-before,SDMT-2y,T25FW-before,T25FW-2y,T25FW-after_2y,T-before,T-after,P-before,P-after,N-before,N-after,DS,DS_L,DS_R,SLEC_before,SLEC_after,SES_after,SES_before,VAA,BDI-before,BDI-after,EDSS-before,EDSS-2y,EDSS-after_2y,KFSS1-Sensory-2y,KFSS1-Sensory-after_2y,KFSS1-Sensory-before,KFSS1-Brain-2y,KFSS1-Brain-after_2y,KFSS1-Brain-before,KFSS1-Bowel-2y,KFSS1-Bowel-after_2y,KFSS1-Bowel-before,KFSS1-Pyramidal-2y,KFSS1-Pyramidal-after_2y,KFSS1-Pyramidal-before,KFSS1-Cerebral-2y,KFSS1-Cerebral-after_2y,KFSS1-Cerebral-before,KFSS1-Visual-2y,KFSS1-Visual-after_2y,KFSS1-Visual-before,KFSS1-Cerebellar-2y,KFSS1-Cerebellar-after_2y,KFSS1-Cerebellar-before,KFSS_M-2y,KFSS_M-after_2y,KFSS_M-before,KFSS_P-2y,KFSS_P-after_2y,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,M_R36-SF12-after,P_R36-SF12-after,R36-SF12-after_Ind
0,MSOAC/0014,46.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSOAC/0016,NaN,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,Y,SPMS,1,1,0,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.55,6.60,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MSOAC/0019,44.0,M,NON-WHITE,NaN,NaN,NaN,NaN,NaN,PPMS,1,1,0,0,NaN,0.0,23.65,21.30,20.15,34.5,35.5,43.0,43.5,51.0,53.0,NaN,NaN,6.30,6.15,5.85,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,3.50,3.0,0.333333,0.166667,0.500000,0.2,0.0,0.2,0.000000,0.166667,0.083333,0.333333,0.5,0.416667,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.2,0.0,0.185185,0.185185,0.240741,0.166667,0.083333,0.208333,0.828571,0.772152,1.0,0.857143,0.721519,1.0
3,MSOAC/0024,60.0,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,NaN,SPMS,1,1,1,1,NaN,0.0,34.45,37.50,NaN,55.0,54.0,NaN,60.0,60.0,NaN,NaN,NaN,4.50,5.25,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.031746,0.023810,4.00,3.75,NaN,0.333333,NaN,0.333333,0.0,NaN,0.1,0.583333,NaN,0.666667,0.166667,NaN,0.250000,0.0,NaN,0.0,0.000000,NaN,0.083333,0.2,NaN,0.5,0.129630,NaN,0.240741,0.291667,NaN,0.375000,0.885714,0.569620,1.0,0.857143,0.716216,1.0
4,MSOAC/0030,28.0,F,WHITE,EUROPE,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,16.55,17.90,NaN,NaN,NaN,NaN,58.0,60.0,NaN,63.5,69.0,4.85,4.70,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,26.0,24.0,1.25,1.25,NaN,0.063492,0.039683,2.00,1.50,NaN,0.166667,NaN,0.166667,0.2,NaN,0.2,0.166667,NaN,0.166667,0.166667,NaN,0.333333,0.0,NaN,0.2,0.166667,NaN,0.083333,0.0,NaN,0.1,0.111111,NaN,0.203704,0.166667,NaN,0.125000,0.933333,0.846154,0.0,0.833333,0.730769,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,MSOAC/9986,46.0,M,WHITE,OCEANIA,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,19.35,18.95,NaN,NaN,NaN,NaN,58.0,60.0,NaN,51.0,60.0,3.90,3.80,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,36.0,35.0,1.25,1.25,NaN,0.047619,0.063492,2.75,2.50,NaN,0.333333,NaN,0.166667,0.0,NaN,0.0,0.333333,NaN,0.250000,0.166667,NaN,0.333333,0.0,NaN,0.0,0.000000,NaN,0.000000,0.0,NaN,0.2,0.111111,NaN,0.148148,0.166667,NaN,0.125000,0.833333,0.730769,0.0,0.800000,0.750000,0.0
2461,MSOAC/9987,18.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [6]:
#variables = ['KFSS_M-2y', 'EDSS-2y', 'T25FW-2y', 'NRELAP']# removed KFSS_P-2y for now -- ('SMSTDY' gave a score of -0.03)
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'EDSS-after_2y', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'NRELAP', 'CESEV']

# Extract targets
#targets = data[variables]

# Extract features by dropping the target columns
#features = data.drop(variables, axis=1)

columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE', 
                    'NHPT-before', 'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'T25FW-before', 'SLEC_before','SES_before',
                    'BDI-before', 'EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before',
                	'P_R36-SF12-before', 'R36-SF12-before_Ind', 'T-before','P-before','N-before']
# still need to change in OE dataframe the SLEC and SES so name is consistent with the others

features = data[columns_to_keep]
features

,AGE,SEX,RACE,CONTINENT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before
0,46.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,M,WHITE,NORTH AMERICA,SPMS,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,44.0,M,NON-WHITE,NaN,PPMS,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN
3,60.0,M,WHITE,NORTH AMERICA,SPMS,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0
4,28.0,F,WHITE,EUROPE,RRMS,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.0,M,WHITE,OCEANIA,RRMS,1,1,0,1,19.35,NaN,58.0,51.0,3.90,36.0,1.25,0.047619,2.75,0.148148,0.125000,0.833333,0.730769,0.0,0.0,0.0,0.0
2461,18.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2462,38.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2463,40.0,F,WHITE,NaN,PPMS,0,1,0,1,23.80,21.5,31.5,NaN,6.15,NaN,NaN,NaN,4.50,0.481481,0.166667,0.728571,0.658228,1.0,0.0,0.0,NaN


In [7]:
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)
features.head()

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS
0,46.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,1,0
1,NaN,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,1,0,1,0,0,0,1,0,0,0,0,1
2,44.0,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN,0,1,1,0,0,0,0,0,0,0,1,0,0
3,60.0,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0,0,1,0,1,0,0,0,1,0,0,0,0,1
4,28.0,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,0,0,1,0


In [8]:
#variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 
             #'M_R36-SF12-after', 'SES_after', 'EDSS-after_2y', 'NRELAP', 'CESEV']

pairs_or_groups = [['EDSS-2y', 'EDSS-after_2y'], ['KFSS_M-2y', 'EDSS-2y'], ['KFSS_P-2y', 'EDSS-2y']]  # Specify the pairs or groups
order_constraint = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y']
num_permutations = 5  # Specify how many random permutations you want
random_state = 42

random_permutations = generate_permutations_with_order(variables, pairs_or_groups, num_permutations, order_constraint, random_state)

# Print the original order followed by all the random permutations
for idx, perm in enumerate(random_permutations, start=0):
    print(f"Permutation {idx}: {', '.join(perm)}")

Permutation 0: KFSS_M-2y, KFSS_P-2y, EDSS-2y, T25FW-2y, NHPT-2y, P_R36-SF12-after, M_R36-SF12-after, SES_after, EDSS-after_2y, NRELAP, CESEV
Permutation 1: KFSS_M-2y, KFSS_P-2y, NHPT-2y, EDSS-2y, T25FW-2y, P_R36-SF12-after, NRELAP, CESEV, M_R36-SF12-after, EDSS-after_2y, SES_after
Permutation 2: KFSS_P-2y, NHPT-2y, KFSS_M-2y, T25FW-2y, EDSS-2y, SES_after, NRELAP, M_R36-SF12-after, CESEV, P_R36-SF12-after, EDSS-after_2y
Permutation 3: KFSS_M-2y, NHPT-2y, KFSS_P-2y, EDSS-2y, T25FW-2y, CESEV, M_R36-SF12-after, SES_after, EDSS-after_2y, NRELAP, P_R36-SF12-after
Permutation 4: NHPT-2y, KFSS_P-2y, KFSS_M-2y, T25FW-2y, EDSS-2y, M_R36-SF12-after, P_R36-SF12-after, NRELAP, SES_after, EDSS-after_2y, CESEV
Permutation 5: T25FW-2y, KFSS_P-2y, KFSS_M-2y, EDSS-2y, NHPT-2y, EDSS-after_2y, NRELAP, P_R36-SF12-after, SES_after, M_R36-SF12-after, CESEV


In [9]:
ordered_targets = random_permutations[0]

In [10]:
#targets = data[ordered_targets] ### CHANGE VAR LIST HERE
#targets

In [11]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

In [12]:
# Generate CV folds
cv=missingness_stratified_cv(features, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

Is it a problem that not all folds have the exact same number?

---

## Chain with *predicted* values propagated

In [13]:
y_pred_chains = []
y_test_list = [[] for _ in range(N_FOLDS)]  # Initialize y_test_list with empty lists for each fold index

# Iterate over each chain ordering
for ordered_targets_chain in random_permutations:
    y_pred_list_chain = []  # List to store predictions for this chain
    
    features_cv = pd.merge(features, pd.DataFrame(cv), left_index=True, right_index=True)
    targets_cv = pd.merge(data[ordered_targets_chain], pd.DataFrame(cv), left_index=True, right_index=True)

    # Fit and predict for each fold for this chain
    for i in range(0, N_FOLDS): 
        Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
        Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
        yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
        yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
        y_test_list_chain = []  # List to store y_test for this fold

        chain = Chain(
            model_reg=RandomForestRegressor(random_state=random_state),
            model_clf=RandomForestClassifier(random_state=random_state),
            propagate="pred",
        )
        chain.fit(Xi_train, yi_train, target_types=None)
        y_pred = chain.predict(Xi_test)
        y_pred_list_chain.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))

        # Append yi_test to the corresponding fold index in y_test_list
        y_test_list[i].append(yi_test)  # Append yi_test for fold i

    y_pred_chains.append(y_pred_list_chain)
    print("Permutation done")

Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done


In [14]:
#y_pred_chains[1][0]

In [15]:
#y_test_list[2][0]

In [16]:
transposed_list = list(zip(*y_pred_chains))

# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
# Now, create a list of lists from these tuples
reorganized_list = [list(df_tuple) for df_tuple in transposed_list]

In [17]:
#reorganized_list[0][1]

In [18]:
transposed_test_list = list(zip(*y_test_list))

# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
# Now, create a list of lists from these tuples
reorganized_test_list = [list(df_tuple) for df_tuple in transposed_test_list]

In [19]:
#reorganized_test_list[0][3]

In [20]:
# Define a function to reorder columns of dataframes
def reorder_columns(dataframes):
    # Get the column order from the first dataframe
    column_order = dataframes[0].columns
    # Reorder columns for each dataframe in the list
    reordered_dataframes = [df[column_order] for df in dataframes]
    return reordered_dataframes

# Apply the function to each list in reorganized_list
reordered_reorganized_list = [reorder_columns(dataframes) for dataframes in reorganized_list]

In [21]:
#reordered_reorganized_list[0][0]

In [22]:
def average_dataframes(dataframes):
    # Concatenate dataframes within the list
    concatenated_df = pd.concat(dataframes)
    # Group by index and calculate the mode for object columns and mean for other types
    averaged_df = concatenated_df.groupby(concatenated_df.index).agg(lambda x: x.mode()[0] if x.dtype == 'O' else x.mean())
    return averaged_df

# Apply the function to each list in reordered_reorganized_list
averaged_dataframes_list = [average_dataframes(dataframes) for dataframes in reordered_reorganized_list]

In [23]:
#averaged_dataframes_list[2]

In [24]:
y_pred_list = averaged_dataframes_list.copy()
y_test_list = reorganized_test_list[0]

y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [25]:
# Initialize a list to store scores
scores = []
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(y_test_cv)):
        y_test = y_test_cv[fold_index][variables.index(variable_name)] 
        y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
        
        # Check if the target variable is numerical or categorical
        if y_test.dtype.kind in 'bifc':
            score = r2_score(y_test, y_pred)
        else:
            score = accuracy_score(y_test, y_pred)
                  
        variable_scores.append(score)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print(f"Scores for each outcome (ensemble with {int(len(random_permutations))} chains - propagate predicted values)")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Scores for each outcome (ensemble with 6 chains - propagate predicted values)
KFSS_M-2y: 0.81 (± 0.02)
KFSS_P-2y: 0.75 (± 0.03)
EDSS-2y: 0.88 (± 0.02)
T25FW-2y: 0.71 (± 0.07)
NHPT-2y: 0.58 (± 0.17)
P_R36-SF12-after: 0.69 (± 0.05)
M_R36-SF12-after: 0.57 (± 0.02)
SES_after: 0.69 (± 0.04)
EDSS-after_2y: 0.74 (± 0.05)
NRELAP: 0.64 (± 0.02)
CESEV: 0.49 (± 0.03)


---

## Chain with *true* values propagated

In [26]:
y_pred_chains = []
y_test_list = [[] for _ in range(N_FOLDS)]  # Initialize y_test_list with empty lists for each fold index

# Iterate over each chain ordering
for ordered_targets_chain in random_permutations:
    y_pred_list_chain = []  # List to store predictions for this chain
    
    features_cv = pd.merge(features, pd.DataFrame(cv), left_index=True, right_index=True)
    targets_cv = pd.merge(data[ordered_targets_chain], pd.DataFrame(cv), left_index=True, right_index=True)

    # Fit and predict for each fold for this chain
    for i in range(0, N_FOLDS): 
        Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
        Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
        yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
        yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
        y_test_list_chain = []  # List to store y_test for this fold

        chain = Chain(
            model_reg=RandomForestRegressor(random_state=random_state),
            model_clf=RandomForestClassifier(random_state=random_state),
            propagate="true",
        )
        chain.fit(Xi_train, yi_train, target_types=None)
        y_pred = chain.predict(Xi_test)
        y_pred_list_chain.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))

        # Append yi_test to the corresponding fold index in y_test_list
        y_test_list[i].append(yi_test)  # Append yi_test for fold i

    y_pred_chains.append(y_pred_list_chain)
    print("Permutation done")

Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done


In [27]:
transposed_list = list(zip(*y_pred_chains))

# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
# Now, create a list of lists from these tuples
reorganized_list = [list(df_tuple) for df_tuple in transposed_list]

In [28]:
transposed_test_list = list(zip(*y_test_list))

# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
# Now, create a list of lists from these tuples
reorganized_test_list = [list(df_tuple) for df_tuple in transposed_test_list]

In [29]:
# Apply the function to each list in reorganized_list
reordered_reorganized_list = [reorder_columns(dataframes) for dataframes in reorganized_list]

In [30]:
# Apply the function to each list in reordered_reorganized_list
averaged_dataframes_list = [average_dataframes(dataframes) for dataframes in reordered_reorganized_list]

In [31]:
y_pred_list = averaged_dataframes_list.copy()
y_test_list = reorganized_test_list[0]

y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [32]:
# Initialize a list to store scores
scores = []
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(y_test_cv)):
        y_test = y_test_cv[fold_index][variables.index(variable_name)] 
        y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
        
        # Check if the target variable is numerical or categorical
        if y_test.dtype.kind in 'bifc':
            score = r2_score(y_test, y_pred)
        else:
            score = accuracy_score(y_test, y_pred)
                  
        variable_scores.append(score)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print(f"Scores for each outcome (ensemble with {int(len(random_permutations))} chains - propagate true values)")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Scores for each outcome (ensemble with 6 chains - propagate true values)
KFSS_M-2y: 0.81 (± 0.02)
KFSS_P-2y: 0.75 (± 0.03)
EDSS-2y: 0.88 (± 0.02)
T25FW-2y: 0.71 (± 0.08)
NHPT-2y: 0.58 (± 0.17)
P_R36-SF12-after: 0.69 (± 0.05)
M_R36-SF12-after: 0.57 (± 0.02)
SES_after: 0.69 (± 0.05)
EDSS-after_2y: 0.76 (± 0.04)
NRELAP: 0.64 (± 0.02)
CESEV: 0.56 (± 0.02)
